# Pokemon cards predictor

### Importacion de librerias


In [1]:
# Import Required Libraries
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageEnhance, ImageFilter

## Creación de datos

### Establecemos el output directory

Usamos `os.makedirs()` para crear el directorio de salida si no existe.

In [2]:
# Set Up Output Directory
output_dir = 'output_images'
os.makedirs(output_dir, exist_ok=True)

### Generamos las variaciones de las imágenes
Iteramos sobre las imágenes de los pokemones y generamos las variaciones de las imágenes. Para ello, usamos la función `generate_variations()` que recibe la imagen y y el id de la carta.

In [2]:
# Read the input data
df = pd.read_csv('./data/cards_with_local_paths.csv')

In [ ]:
# Función para generar variaciones de una imagen
def generate_variations(image_path, image_id):
    variations = []
    image = Image.open(image_path)
    
    # Incluir la imagen original
    variations.append((image, f"{image_id}_original"))
    
    # 1. Blanco y negro
    bw = image.convert("L")
    variations.append((bw, f"{image_id}_bw"))
    
    # 2. Desenfoque
    blurred = image.filter(ImageFilter.GaussianBlur(2))
    variations.append((blurred, f"{image_id}_blurred"))
    
    # 3. Más saturación
    enhancer = ImageEnhance.Color(image)
    saturated = enhancer.enhance(1.5)
    variations.append((saturated, f"{image_id}_saturated"))
    
    # 4. Menos saturación
    desaturated = enhancer.enhance(0.5)
    variations.append((desaturated, f"{image_id}_desaturated"))
    
    # 5. Inclinada a la izquierda
    rotated_left = image.rotate(15)
    variations.append((rotated_left, f"{image_id}_rotated_left"))
    
    # 6. Inclinada a la derecha
    rotated_right = image.rotate(-15)
    variations.append((rotated_right, f"{image_id}_rotated_right"))
    
    # 7. Zoom in
    zoom_in = image.resize((int(image.width * 1.2), int(image.height * 1.2)))
    variations.append((zoom_in, f"{image_id}_zoom_in"))
    
    # 8. Zoom out
    zoom_out = image.resize((int(image.width * 0.8), int(image.height * 0.8)))
    variations.append((zoom_out, f"{image_id}_zoom_out"))
    
    # 9. Con ruido
    noise = image.copy()
    noise = noise.convert("RGB")
    pixels = noise.load()
    for i in range(noise.size[0]):
        for j in range(noise.size[1]):
            r, g, b = pixels[i, j]
            noise_factor = 25
            r = int(r + noise_factor * (0.5 - os.urandom(1)[0] / 255))
            g = int(g + noise_factor * (0.5 - os.urandom(1)[0] / 255))
            b = int(b + noise_factor * (0.5 - os.urandom(1)[0] / 255))
            pixels[i, j] = (r, g, b)
    variations.append((noise, f"{image_id}_noise"))
    
    # 10. Brillo aumentado
    enhancer = ImageEnhance.Brightness(image)
    bright = enhancer.enhance(1.5)
    variations.append((bright, f"{image_id}_bright"))
    
    # 11. Rotación aleatoria
    random_rotation = image.rotate(np.random.uniform(-30, 30))
    variations.append((random_rotation, f"{image_id}_random_rotation"))
    
    # 12. Traslación
    translated = image.transform(image.size, Image.AFFINE, (1, 0, 10, 0, 1, 10))
    variations.append((translated, f"{image_id}_translated"))
    
    # 13. Corte (Crop)
    width, height = image.size
    crop_area = (10, 10, width - 10, height - 10)
    cropped = image.crop(crop_area)
    variations.append((cropped, f"{image_id}_cropped"))
    
    # 14. Espejo horizontal
    flipped_horizontal = image.transpose(Image.FLIP_LEFT_RIGHT)
    variations.append((flipped_horizontal, f"{image_id}_flipped_horizontal"))
    
    # 15. Contraste aumentado
    enhancer = ImageEnhance.Contrast(image)
    high_contrast = enhancer.enhance(1.5)
    variations.append((high_contrast, f"{image_id}_high_contrast"))
    
    return variations

In [5]:
# Generamos las variaciones para las primeras 102 imágenes (Primera generación de cartas pokemon)
variations_data = []

for index, row in df.head(102).iterrows():
    image_path = row['local_image_path']
    image_id = row['id']
    variations = generate_variations(image_path, image_id)
    for var_image, var_name in variations:
        var_image_path = os.path.join(output_dir, f"{var_name}.png")
        var_image.save(var_image_path)
        variations_data.append({'id': image_id, 'local_image_path': var_image_path})

# Create a DataFrame with the variations
variations_df = pd.DataFrame(variations_data)

# Save the DataFrame to a CSV file
variations_df.to_csv('cards_with_variations.csv', index=False)

### Creamos un nuevo DataFrame para almacenar los datos de las variaciones


In [6]:
# Crear un DataFrame con las variaciones
variations_df = pd.DataFrame(variations_data)

### Guardamos el DataFrame con las variaciones en un archivo CSV
Guardamos el DataFrame con las variaciones en un archivo CSV usando `to_csv()`.

In [7]:
# Save the DataFrame with variations to a CSV file using to_csv()
variations_df.to_csv('cards_with_variations.csv', index=False)

## Entranamiento del modelo

### Entrenamiento con red neuronal

#### Importación de librerías

In [ ]:
# Importación de librerias
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

KeyboardInterrupt: 

#### Carga de datos

In [ ]:
# Leer el dataframe
df = pd.read_csv('cards_with_variations.csv')

# Preparar los datos
image_size = (128, 128)  # Tamaño al que redimensionaremos las imágenes
num_classes = df['id'].nunique()  # Número de clases (IDs únicos)

# Función para cargar y procesar las imágenes
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Convertir a RGB
    image = image.resize(image_size)
    image = np.array(image)
    return image

# Cargar las imágenes y las etiquetas
images = np.array([load_image(row['local_image_path']) for _, row in df.iterrows()])
labels = to_categorical(df['id'].astype('category').cat.codes, num_classes=num_classes)

NameError: name 'pd' is not defined

#### División de los datos en entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

#### Entrenamiento

In [13]:

# Construir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(image_size[0], image_size[1], 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
batch_size = 64
epochs = 50

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss:.2f}, Accuracy: {accuracy*100:.2f}%')

Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 417ms/step - accuracy: 0.0111 - loss: 30.8460 - val_accuracy: 0.0856 - val_loss: 4.3850
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 446ms/step - accuracy: 0.1251 - loss: 4.0239 - val_accuracy: 0.5352 - val_loss: 2.4734
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.5134 - loss: 2.1238 - val_accuracy: 0.8471 - val_loss: 0.6389
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 424ms/step - accuracy: 0.7915 - loss: 0.7499 - val_accuracy: 0.9297 - val_loss: 0.3378
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 429ms/step - accuracy: 0.8998 - loss: 0.3866 - val_accuracy: 0.9235 - val_loss: 0.3227
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 420ms/step - accuracy: 0.9195 - loss: 0.3634 - val_accuracy: 0.9174 - val_loss: 0.3510
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 437ms/step - accuracy: 0.9216 - loss: 0.2942 - val_accuracy: 0.9450 - val_loss: 0.2709
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.9497 - loss: 0.1869 - val_accuracy

#### Guardado del modelo entrenado con red neuronal

In [15]:
# Guardar el modelo
model.save('pokemon_card_predictor.keras')

#### Testeo del modelo

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import pandas as pd

# Cargar el modelo
model = load_model('pokemon_card_predictor.keras')

# Función para cargar y procesar una imagen
def load_and_preprocess_image(image_path, image_size=(128, 128)):
    image = Image.open(image_path)
    image = image.resize(image_size)
    image = np.array(image)
    if image.shape[-1] == 4:  # Si la imagen tiene un canal alfa, eliminarlo
        image = image[..., :3]
    image = image / 255.0  # Normalizar la imagen
    return image

# Función para predecir el ID de una carta
def predict_card_id(image_path, model, image_size=(128, 128)):
    image = load_and_preprocess_image(image_path, image_size)
    image = np.expand_dims(image, axis=0)  # Añadir una dimensión para el batch
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0]

# Cargar el DataFrame para obtener el mapeo de IDs
df = pd.read_csv('cards_with_variations.csv')
id_to_label = {i: label for i, label in enumerate(df['id'].astype('category').cat.categories)}

# Probar el modelo con una nueva imagen
test_image_path = 'output_images/dp1-1_saturated.png'  # Reemplaza con la ruta de tu imagen de prueba
predicted_class = predict_card_id(test_image_path, model)
predicted_label = id_to_label[predicted_class]

print(f'Predicted ID: {predicted_label}')

C:\Users\d.ramirez.vaquero\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
Predicted ID: dp1-1


### Entrenamiento con Random Forest

#### Importación de librerías

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#### Carga de datos

In [5]:
# Leer el dataframe
df = pd.read_csv('./data/cards_with_variations.csv')

# Obtener las 102 primeras cartas únicas
unique_cards = df['id'].unique()[:102]
df = df[df['id'].isin(unique_cards)]

# Preparar los datos
image_size = (128, 128)  # Tamaño al que redimensionaremos las imágenes
label_encoder = LabelEncoder()
df['id'] = label_encoder.fit_transform(df['id'])  # Convertir etiquetas a formato numérico
num_classes = df['id'].nunique()  # Número de clases (IDs únicos)

print(f"Total de imágenes: {len(df)}")
print(f"Total de clases: {num_classes}")

# Función para cargar y procesar las imágenes
def load_image(image_path):
    try:
        image = Image.open(image_path).convert('RGB')  # Convertir a RGB
        image = image.resize(image_size)
        image = np.array(image)
        return image
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Listas para almacenar las imágenes y etiquetas
images = []
labels = []

# Procesar y guardar las imágenes y etiquetas en listas
for i, row in tqdm(df.iterrows(), total=len(df), desc="Procesando imágenes"):
    image = load_image(row['local_image_path'])
    if image is None:
        # Intentar cargar la imagen desde la carpeta de fallos
        print(f"Failed to process image {row['local_image_path']}. Trying to load from failures folder.")
        failure_image_path = os.path.join('output_images/failures', os.path.basename(row['local_image_path']))
        image = load_image(failure_image_path)
        print("Succesfully loaded image from failures folder.")
    
    if image is not None:
        images.append(image)
        labels.append(row['id'])
    else:
        print(f"Failed to process image {row['local_image_path']} and {failure_image_path}")

# Convertir las listas a arrays de numpy
images = np.array(images)
labels = np.array(labels)

print("Imágenes y etiquetas procesadas guardadas en variables")

Total de imágenes: 1632
Total de clases: 102


Procesando imágenes: 100%|██████████| 1632/1632 [07:14<00:00,  3.76it/s]

Imágenes y etiquetas procesadas guardadas en variables


#### Prepardo de los datos

In [6]:
# Aplanar las imágenes para que puedan ser utilizadas por el clasificador
images_flattened = images.reshape(images.shape[0], -1)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(images_flattened, labels, test_size=0.2, random_state=42)


#### Entrenamiento

In [7]:
# Entrenar el modelo de Bosques Aleatorios
print("Entrenando el modelo de Bosques Aleatorios...")
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

Entrenando el modelo de Bosques Aleatorios...


RandomForestClassifier(random_state=42)

#### Evaluación

In [8]:
print("Evaluando el modelo...")
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Evaluando el modelo...
Accuracy: 96.33%


#### Guardado del modelo

In [9]:
# Guardar el modelo
import joblib
joblib.dump(clf, 'pokemon_card_classifier.pkl')

['pokemon_card_classifier.pkl']